# Uvod

Nama relevantan primer je primer prevodioca, i to za početak sa srpskog jezika na srpski znakovni jezik. S obzirom da takva baza ne postoji, ovde ću koristiti neki generic primer, ali ću objašnjavati šta bi trebalo da se dešava u našem slučaju. 

S obzirom da ja paralelno učim i pišem ovo, probaću da objašnjavam tačno šta radi python kod nezavisno od konteksta transformera.

Takođe, probaću da sve što je moguće i teorijski objasnim. Znanje neuralnih mreža je korisno, ali potrudiću se da ne bude neophodno.

To znači da svako parče koda ima 3 dela:

1. Sve nove komande koje do sad nisu viđene, a korisno je da se zna šta rade generalno;

2. Šta kod postiže;

3. Teoretska osnova za taj deo.

Ispod je napisano sve neophodno da se kod izvršava.

In [ ]:
pip install -r requirements.txt

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset as TorchDataset
from torch.utils.tensorboard import SummaryWriter

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from datasets import Dataset as HFDataset
from translate.storage.tmx import tmxfile

import warnings
import torchmetrics

from pathlib import Path
from tqdm import tqdm
import os

import math
import tabulate

from typing import Tuple, Dict, Any, List
from collections.abc import Callable

# Transformer

Transformer je arhitektura za deep learning (gde deep znači da se koristi više slojeva u mreži, a sve se radi na neuralnim mrežama), koja je dizajnirana u radu [Attention is all you need](https://arxiv.org/pdf/1706.03762). Sastoji se od encoder-a i decoder-a. 

Tekst se konvertuje u tokene, koji se zatim konvertuju u vektore (što se zove embedding), u nekom višedimenzionom vektorskom prostoru. Početni embedding je klasičan dictionary, gde svaki token iz rečnika ima svoj embedding.

U svakom sloju, suština je da svaki embedding primi kontekst od drugih tokena, i samim tim update-uje vrednosti za svoj embedding. Ovime se token u ovom vektorskom prostoru sve više i više približava svojoj najprigodnijoj lokaciji.

U nastavku pratimo šta se dešava prilikom treninga transformera.

## Baza podataka

Za početak, moramo da definišemo bazu podataka sa kojom će naš model da radi. Ona se nadovezuje na `torch` bazu, i implementira njene tri glavne metode: `__init__(dataset, source_tokenizer, target_tokenizer, source_language, target_language, context_size)`, `__len__()` i `__getitem__(index)`, koje redom inicijalizuju objekat, vraćaju njegovu veličinu (to jest broj podataka - za nas parova rečenica) i vraćaju konkretan podatak sa datog indeksa.

Uz to, unapred moramo da definišemo još jednu funkciju, a koja će biti objašnjena kasnije. To je funkcija `causal_mask(size)`.

In [3]:
def causal_mask(size: int) -> torch.Tensor:
    
    mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int)
    return mask == 0

class BilingualDataset(TorchDataset):

    def __init__(
            self, 
            dataset: HFDataset, 
            source_tokenizer: Tokenizer, 
            target_tokenizer: Tokenizer, 
            source_language: str, 
            target_language: str, 
            context_size: int
        ) -> None:
        super().__init__()

        self.context_size = context_size

        self.dataset = dataset

        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer

        self.source_language = source_language
        self.target_language = target_language
        
        self.sos_token = torch.tensor([source_tokenizer.token_to_id('[SOS]')], dtype = torch.int64)
        self.eos_token = torch.tensor([source_tokenizer.token_to_id('[EOS]')], dtype = torch.int64)
        self.pad_token = torch.tensor([source_tokenizer.token_to_id('[PAD]')], dtype = torch.int64)


    def __len__(self) -> int:
        return len(self.dataset)
    
    
    def __getitem__(
            self, 
            index: int
        ) -> Dict[str, Any]:
        source_target_pair = self.dataset[index]

        source_text = source_target_pair['translation'][self.source_language]
        target_text = source_target_pair['translation'][self.target_language]

        encoder_input_tokens = self.source_tokenizer.encode(source_text).ids
        decoder_input_tokens = self.target_tokenizer.encode(target_text).ids

        encoder_num_padding_tokens = self.context_size - len(encoder_input_tokens) - 2
        decoder_num_padding_tokens = self.context_size - len(decoder_input_tokens) - 1
        
        if encoder_num_padding_tokens < 0 or decoder_num_padding_tokens < 0:
            raise ValueError("Sentence is too long!")
        
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(encoder_input_tokens, dtype = torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * encoder_num_padding_tokens, dtype = torch.int64)
            ],
            dim = 0
        )

        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(decoder_input_tokens, dtype = torch.int64),
                torch.tensor([self.pad_token] * decoder_num_padding_tokens, dtype = torch.int64)
            ],
            dim = 0
        )

        label = torch.cat(
            [
                torch.tensor(decoder_input_tokens, dtype = torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * decoder_num_padding_tokens, dtype = torch.int64)
            ],
            dim = 0
        )

        assert encoder_input.size(0) == self.context_size
        assert decoder_input.size(0) == self.context_size
        assert label.size(0) == self.context_size

        return {
            "encoder_input": encoder_input,
            "decoder_input": decoder_input,
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
            "label": label,
            "source_text" : source_text,
            "target_text" : target_text
        }

Primećujemo da vraćanje elementa iz baze vraća više stvari. Sve one će biti objašnjene naknadno. To su:

1. encoder_input: Ulaz u encoder modela;

2. decoder_input: Ulaz u decoder modela;

3. encoder_mask: Maska za encoder;

4. decoder_mask: Maska za decoder;

5. label: Očekivani izlaz iz transformer-a;

6. source_text: Rečenica na originalnom jeziku;

7. target_text: Prevedena rečenica.

## Encoder

Encoder je deo transformer-a koji se odnosi na (u slučaju prevodioca) procesiranje ulazne rečenice. Isključivo u ovom delu će model naučiti (nadamo se) neku formu jezika i uspeti da razume odnose između reči.

### Tokenizer

Cilj prevodioca je da primi rečenicu na jednom jeziku i izbaci istu tu rečenicu na drugom jeziku. Za nas je rečenica uređena lista reči, a za kompjuter - niz tokena. Tokeni su, u generalnom slučaju, osnovni delovi građe teksta. Ne moraju biti reči u ljudskom shvatanju, iako se mogu i tako postaviti. 

U nekoj arhitekturi je moguće smatrati da je reč "obaveštavamo" jedan token, a u drugoj bi to mogla biti i dva tokena "obaveštava" + "mo". Prednosti postoje za obe situacije, a posmatra se ušteda vremena i mogućnost boljeg razumevanja teksta.

Međutim, nisu reči jedini sastavni deo teksta. Postoje i neke stvari koje ljudi uzimaju zdravo za gotovo, a to su definitivno početak i kraj rečenice, interpukcijski znakovi, činjenica da neke reči imaju različito značenje u zavisnosti od toga da li im je prvo slovo veliko ili malo, i tako dalje. Sve ovo je neophodno uzeti u obzir.

Dakle, prvi korak je prebaciti tekst koji želimo u tokene.

Tokenizer je deo programa koji listu svih rečenica koje imamo u bazi (dakle onih sa kojima naš transformer treba da radi) pretvara u rečnik tokena. Svakom tokenu koji se nađe u ovim rečenicama dodeljuje jedinstveni id.

Takođe, postoje $4$ specijalna tokena, a to su:

1. ***[UNK]*** token koji će (u korišćenju i treniranju modela) menjati one tokene koji nemaju mesto u rečniku. To su ili tokeni kojih nema u bazi, ili oni sa nedovoljnim brojem pojavljivanja.

2. ***[PAD]*** token koji služi da rečenicu koja nije dovoljno dugačka dopuni do kraja. Transformer radi sa rečenicama konstantne dužine (u tokenima), pa da bi bio upotrebljiviji, ukoliko ima manje tokena od te dužine, dopuni se ovim tokenima.

3. ***[SOS]*** i ***[EOS]*** tokeni koji označavaju početak i kraj rečenice.

Dakle, svaka rečenica će nakon tokenizer-a da ima izgled:

$$\left[\text{SOS}\right]\ T_1\ T_2\ T_3\ \dots\ T_K\ \left[\text{EOS}\right]\ \left[\text{PAD}\right]\ \left[\text{PAD}\right]\ \dots\ \left[\text{PAD}\right]$$

gde će ukupan broj tokena uvek da bude isti, i zvaćemo ga `context_size`.

Pre definicije tokenizer-a, treba nam i fajl koji će čuvati hiperparametre modela, kao i neke njegove opisne karakteristike (jezici koji se koriste, ime modela, ...). To radimo sa `config`-om, koji pozivamo funkcijom `get_config()`.

In [4]:
def get_config():
    return {
        "batch_size": 64,
        "num_epochs": 100,
        "learning_rate": 3 * 10**-4,
        "context_size": 64,
        "model_dimension": 128,
        "source_language": "en",
        "target_language": "asl",
        "model_folder": "weights",
        "model_basename": "english_to_gloss_",
        "preload": None,
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/english_to_gloss",
        "seed": 561
    }

Sada možemo da pređemo i na učitavanje tokenizer-a.

In [5]:
def get_all_sentences(
        dataset: HFDataset,
        language: str
    ):
    for item in dataset:
        yield item['translation'][language]

def get_or_build_tokenizer(
        config, 
        dataset: HFDataset, 
        language: str,
        force_rewrite: bool = False,
        min_frequency: int = 5,
        vocab_size: int = 1000000
    ) -> Tokenizer:
    tokenizer_path = Path(config['tokenizer_file'].format(language))

    if not Path.exists(tokenizer_path) or force_rewrite:

        tokenizer = Tokenizer(WordLevel(unk_token = '[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()

        trainer = WordLevelTrainer(
            special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"], 
            min_frequency = min_frequency, 
            vocab_size = vocab_size
            )
        
        tokenizer.train_from_iterator(get_all_sentences(dataset, language), trainer = trainer)

        tokenizer.save(str(tokenizer_path))

    else: 
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    print(f"Number of tokens in {language} is {tokenizer.get_vocab_size()}.")
    return tokenizer

Tokenizer koji je napravljen u kodu za tokene ima reči i interpukcijske znakove. Ukoliko je moguće, biće učitan iz odgovarajućeg .json fajla, a u suprotnom, rečenice će biti procesuirane jedna po jedna, a na kraju, svi tokeni koji imaju bar `min_frequency` pojavljivanja u njima, će biti ubačeni u rečnik. Rečnik o kome ovde govorimo ima za key-eve tokene a za value-e ima int-ove koji se ponašaju kao redni broj tokena u rečniku.

### Input embeddings

Nakon dobijanja liste tokena, želimo da dobijemo i njihove vektorske reprezentacije - embeddinge.

Embedding je samo jedan element nekog ogromno-dimenzionog vektorskog prostora. U pomenutom radu koristi se $512$ dimenzija, u ChatGPT 3 se koristi $12288$. Ovaj broj ćemo označavati sa $d_{model}$.

Razlog za veliki broj dimenzija je ogroman broj reči i mogućnosti za njihovo značenje. Dobar primer za ovo je što, u slučaju GPT-a, jedan pravac u ovom $12288$-dimenzionom prostoru označava pol. Zapravo, ako se posmatra razlika vektora kojima odgovaraju muškarac i žena, i ta razlika se sabere sa nekom imenicom ženskog pola, dobiće se približno vektor odgovarajuće imenice muškog pola (*kraljica* $\rightarrow$ *kralj*, *majka* $\rightarrow$ *otac*, itd). Slično, postoje smerovi koji odgovaraju množini, državljanstvu, i sličnim pojmovima. Ovo se naravno odnosi na već istreniran transformer. Naravno da je ovakva interpretabilnost retkost i da su pravci u vektorskom prostoru u velikom broju slučajeva neki pattern-i koji za ljude nemaju smisla.

Takođe, slični tokeni (po značenju) se nalaze blizu jedan drugog po vektorima. Ovo je još jedan razlog zašto je veliki broj dimenzija bitan, jer ima više prostora da se slične stvari nađu bliže međusobno nego sa ostalim, dok još preostaje prostora da se bitno razlikuju i međusobno. Da postoje, na primer, tri dimenzije, praktično ne bi bilo dovoljno prostora da se tako nešto izvede.

In [6]:
class InputEmbeddings(nn.Module):

    def __init__(
            self, 
            model_dimension: int, 
            vocab_size: int
        ) -> None:
        super().__init__()

        self.model_dimension = model_dimension
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, model_dimension)

    def forward(self, x) -> torch.Tensor:
        return self.embedding(x) * math.sqrt(self.model_dimension)

### Positional encoding

Nije dovoljno da se modelu predaju jedino embeddings za sve tokene. Njihov kontekst zavisi i od pozicije u rečenici. 

Positional encoding je standardan način da se preda i ta informacija. Ovo ne zavisi od modela i od tokena, već samo od pozicije i veličine embeddinga.

Formule koje se koriste su
$$PE(\text{position}, 2i) = \sin \frac{\text{position}}{10000 ^ {\frac{2i}{d_{model}}}},$$
za parne pozicije u vektoru, i
$$PE(\text{position}, 2i + 1) = \cos \frac{\text{position}}{10000 ^ {\frac{2i}{d_{model}}}},$$
za neparne.

Ovde se koriste neke bitne funkcije iz `pytorch`-a. To su:

1. `torch.arange(start, end)` - Slična kao standardni python range, uz to što sada vraća jednodimenzionalni tenzor. (Tenzor je generalizacija vektora i matrica na više dimenzija. 1D tenzor je matematički isto što i vektor.)

2. `torch.zeros(*size)` - Tenzor sa svim nulama i zadatim dimenzijama.

3. `torch.tensor.unsqueeze(n)` - Ubacuje još jednu dimenziju u tenzor, na mesto prosleđenog parametra. Ako smo imali tenzor dimenzije $\left(a_0 \times a_1 \times ... \times a_{j - 1}\right)$, onda ovo ubacuje dimenziju (veličine 1) na n-to mesto u ovom nizu, pa tenzor postaje dimenzije $\left(a_0 \times a_1 \times ... \times a_{n - 2} \times 1 \times a_{n - 1} \times ... \times a_{j - 1}\right).$

4. Sve matematičke funkcije u `torch`-u se primenjuju na svaki element tenzora zasebno, i vraća se tenzor iste dimenzije.

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(
            self, 
            model_dimension: int, 
            context_size: int, 
            dropout: float
        ) -> None:
        super().__init__()

        self.model_dimension = model_dimension
        self.context_size = context_size
        self.dropout = nn.Dropout(dropout)
        
        positional_encodings = torch.zeros(context_size, model_dimension)
        position = torch.arange(0, context_size, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dimension, 2).float() * (-math.log(10000.0) / model_dimension))
        
        positional_encodings[:, 0::2] = torch.sin(position * div_term)
        positional_encodings[:, 1::2] = torch.cos(position * div_term)

        positional_encodings = positional_encodings.unsqueeze(0)

        self.register_buffer('pe', positional_encodings)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

Par bitnih stvari vezanih za kod su sledeće:
1. `register_buffer(name, tensor)` je metod u torch-u koji suštinski postavlja parametre za model koji se neće trenirati. U ovom slučaju, `pe` je samo positional encoding, koji je konstantan.

2. `requires_grad_(bool)` omogućava pamćenje operacija koje su učinjene nad nekim tenzorom, dok ga postavljanje na `False` tera da ih ne uzima u obzir nikad.

Ovde uvek vraćamo samo statičku vrednost za positional encoding, koja se dodaje na već napravljeni input embedding. Ovime dobijamo kodirane vrednosti svih tokena, zajedno sa svojim originalnim značenjem i pozicijom u rečenici.

### Normalizacija

Korak koji će često biti korišćen je normalizacija. Čisto služi kao metod za smirivanje podataka, ne razlikuje se mnogo od standardnih normalizacija.

U ovom slučaju, svaki vektor se normalizuje zasebno (u odnosu na svoju srednju vrednost i disperziju). Pričam ovde vektor jer se radi ne samo na početnim tokenima, već u praktično svakom koraku transformera.

Ukoliko imamo vektor $\left(a_0, a_1, \dots, a_{d_{model} - 1}\right)$, sa srednjom vrednošću $\mu$ i disperzijom $\sigma^2$, normalizacija koja se radi je
$$\hat{x} = \alpha \frac{x - \mu}{\sqrt{\sigma^2 + \varepsilon}} + \beta.$$

Parametri $\alpha$ i $\beta$ služe za ubacivanje podataka u odgovarajući, željeni interval. Parametar $\varepsilon$ služi za numeričku stabilnost. Omogućava da čak iako su vrednosti $\sigma^2$ male, ne dobijemo grešku deljenja sa nulom (što se dešava kada su svi podaci približno isti).

In [8]:
class LayerNormalization(nn.Module):

    def __init__(
            self, 
            features: int, 
            eps: float = 10**-6
        ) -> None:
        super().__init__()

        self.eps = eps

        self.alpha = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))

    def forward(self, x):
        mean = x.mean(dim = -1, keepdim = True)
        std = x.std(dim = -1, keepdim = True)

        return self.alpha * (x - mean) / (std + self.eps) + self.bias

U kodu `bias` igra ulogu prethodno definisanog parametra $\beta$. Inicijalizacija ovih promenljivih kao `nn.Parameter(data)` omogućava da se treniraju.

Sam kod je ništa više nego implementacija formule.

### Attention

Ovo je glavni i najzanimljiviji korak celog transformera. Deo je i enkodera i dekodera, ali na malo drugačiji način, iako je logika ista.

Naziv attention označava (generalno u ML-u) to da se iz podataka izvlače samo bitni, relevantni podaci. Postiže se tako što model uči weight-ove koji daju različit značaj različitim elementima.

#### Single-head self attention

Ovaj pojam je u srcu celog attention bloka. Fokusiramo se na primer. Nakon što se mreža stvarno istrenira, teško da postoji mogućnost da čovek razume šta ti weight-ovi koji su dobijeni stvarno označavaju, ali kroz primer će se videti šta planiramo da postignemo, a kompjuter će sam izvesti svoje razumevanje toga.

Pravimo prevodioca, tako da pretpostavljamo da imamo neku rečenicu na srpskom jeziku koju želimo da prevedemo na srpski znakovni jezik. Neka je ta rečenica ***Brzi voz 271 dolazi na kolosek broj 7.***

U ovoj rečenici postoje dve glavne imenice, voz i kolosek. Pretpostavimo da je željeni rezultat da vidimo koje sve druge reči bliže određuju (zasebno) te imenice. (Takođe, pravimo se da su tokeni reči.) Mi znamo da su to **brzi** za voz i **broj 7** za kolosek.

S obzirom da se bavimo ML-om, jedini način da nešto ovako proverimo je nekakvim operacijama nad matricama i vektorima.

To znači da trenutni embedding za reči *voz* i *brzi* moraju da imaju način da komuniciraju. Reč *voz* kao da pita "Šta me bliže određuje?", dok *brzi* odgovara "Ja te bliže određujem!". 

Ukoliko postoji način da kodiramo to pitanje i odgovor u istodimenzionom vektorskom prostoru, način da se proveri koliko su oni bliski je njihob skalarni proizvod. Što je on veći, odnosno manji, to su vektori bliskiji, odnosno različitiji.

Razlog zašto posmatramo skalarne proizvode neskaliranih vektora je to što hoćemo da njihova dužina zapravo utiče na rezultat. Intuitivno, reči kao *katastrofa*, koje bi imale veliku dužinu kao vektori, i treba da puno utiču na ostale tokene zbog njihove prirode. Intenzitet vektora je, dakle, način da predstavimo intenzitet reči.

Ostaje pitanje kako dobijamo te vektore, kojima postavljamo pitanje i dajemo odgovor. Naravno, množimo trenutne embedding-e nekim matricama.

Proces koji se dešava je sledeći:

1. Imamo trenutne embedding-e (koji uključuju i poziciju u rečenici i originalno značenje reči), koje označavamo sa $E_0, E_1, \dots, E_{N-1}$. To su vektori dužine $d_{model}$. Svi oni su raspoređeni u jednu matricu veličine $N \times d_{model},$ koju ćemo nazivati $I$, a gde su zasebne reči redovi u matrici ($I$ je od Input, jedinična matrica se označava sa $E_n$). U zavisnosti od konteksta, označavaćemo je sa još $Q, K, V$, kada se govori o query-ju, key-u, ili value-u (što ćemo videti šta označava).

2. Imamo neke matrice $W_Q$, i $W_K$ (koje su parametri modela), dimenzija (u opštem slučaju) $d_{model} \times d_{KQ}$, gde je, u slučaju ovog koda $d_{model} = d_{KQ}$, ali generalno je dozvoljeno da se razlikuje. (U slučaju ChatGPT-a je veličine $128$.)

3. Množenjem matrica $Q \times W_Q$ i $K \times W_K$ dobijamo matrice koje označavamo redom sa $Q'$ i $K'$, i koje imaju dimenziju $N \times d_{KQ}$. (Ovde se vidi razlika između encoder-a i decoder-a. U encoder-u, $Q$ i $K$ su iste matrice, dok ćemo u decoder-u dozvoliti da budu različite.)

4. Naravno, $k$-ti red tih dobijenih matrica se odnosi na $k$-tu reč u rečenici. Da bismo našli skalarni proizvod koji nas zanima, uzimamo $2.$ red iz $Q'$, i $1.$ red iz $K'$. Ovo se postiže takođe i uzimanjem proizvoda $Q'K'^{T}$, čime se dobija matrica dimenzija $N \times N$. Na $(k, j)$ poziciji u ovoj matrici nalazi se skalarni proizvod $k$-tog reda iz $Q'$ i $j$-tog reda iz $Q'$.

U gornjim koracima, broj $N$ označava `context_size`.

Sada imamo nešto što označava kako reči međusobno utiču. Pitanje je kako da sad da tom informacijom prođemo kroz reči, i napravimo ih reprezentativnijim za dati kontekst. Za to služi Value matrica.

#### Softmax

Treba da prethodno dobijenu matricu skaliramo tako da dobijemo verovatnoće. Za to koristimo standardnu funkciju **softmax**. Ova funkcija pretvara neki vektor realnih brojeva u funkciju raspodele. Važi:
$$\text{softmax}\left(a_1, a_2, \dots, a_n\right)=\frac{1}{\displaystyle\sum_{i=1}^n e^{a_i}}\left(e^{a_1}, e^{a_2}, \dots, e^{a_n}\right).$$
Ovime dobijamo brojeve veće od nula, a koji se sabiraju do $1$. Primećujemo da postavljanje nekog od $a_i$ na $-\infty$ odgovara tome da će dobijeni vektor na toj poziciji da sadrži nulu.

#### Value matrica

Da bismo dobili rezultat attention-a, izvodimo sledeće korake:

1. Matrica $Q'K'^{T}$ se skalira brojem $\frac{1}{\sqrt{d_k}}$ (u slučaju single head attention-a, broj $d_k$ je isti kao i $d_{model}$, dok se u opštem slučaju računa kao $d_{model} / h$), a zatim na dobijenu matricu primeni funkcija softmax (na redove te matrice!)

2. Imamo matricu $W_V$ (takođe parametar modela) i dimenzije $d_{model} \times d_{model}$. 

3. Množimo matrice $V$ i $W_V$ (koja je ista kao $I$ u slučaju encoder-a), čime dobijamo $V'$, matricu dimenzije $N \times d_{model}$.

4. Množimo rezultat prvog koraka (matrica skalirana a zatim primenjen softmax) i $V'$. Ovime dobijamo i rezultat single head attention-a, matricu veličine $N \times d_{model}.$

Rezultat je matrica iste veličine kao i ona sa kojom smo počeli. Praktično, rezultat se tumači kao novi embedding-ovi, koji su primili kontekst od ostalih reči.

#### Masking u encoder-u

Videli smo malopre da, ako je, na primer, $\text{context size} = 16$, rečenica ima oblik ***[SOS] Brzi voz 271 dolazi na kolosek broj 7 [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]***.

Mi ne želimo da token ***[PAD]*** utiče na reči, jer on zapravo ne postoji, i služi samo da bismo rečenicu dopunili do odgovarajuće dužine.

To postižemo maskiranjem, koje u ovom slučaju zovemo *padded masking*. Rezultate u matrici $Q'K'^{T}$ koji odgovaraju ovim ***[PAD]*** tokenima menjamo sa $-\infty$. Time postižemo da oni ne utiču na krajnji rezultat.

#### Multi-head self attention

Umesto da posmatramo potpune embedding-e za svaki vektor, možemo njih da podelimo na $h$ delova (mora da važi $h \mid d_{model}$, i broj $\frac{d_{model}}{h}$ označavamo sa $d_k$, ili $\text{head dimension}$). 

Ovo deljenje postižemo projekcijom matrica $Q, K, V$ na $h$ dimenzija, i to na sledeći način:

$$Q_i = QW_i^Q$$
$$K_i = KW_i^K$$
$$V_i = VW_i^V$$

Svaka glava potom radi single head attention, na potpuno isti način kao malopre:

$$\text{output}_i = \text{softmax}\left(\frac{Q_iK_i^{T}}{d_k}\right)V_i.$$

Matrice $W_i^{Q}$, $W_i^{K}$ i $W_i^{V}$ su sve dimenzija $d_{model} \times d_k$, što znači da su $Q_i$, $K_i$ i $V_i$ dimenzija $N \times d_k.$

Sada spajamo ove dobijene matrice:

$$O = \text{concat}\left[\text{output}_1, \dots, \text{output}_h\right]$$

i konačno rezultat množimo matricom $W^O$ ($O$ je dimenzija $N \times d_{model}$, a $W^O$ dimenzija $d_{model} \times d_{model}$), i dobijamo matricu koja je konačni rezultat multi head attention-a: $OW^O.$ 

Primetimo da se broj parametara nije promenio od malopre (osim dodate matrice $W^O$). Razlog zašto se ipak koristi multi head attention je taj što tako omogućavamo raznim rečima da nauče različite mogućnosti za uticaj drugih reči na sebe. Što je više glava, to više raznih načina interakcije između reči model može da nauči. Razni $W_i^{K}$ mogu nezavisno da uče različite stvari, za razliku od jedinstvene matrice u slučaju single head attention-a.

Kod je samo implementacija matematičkih formula, uz igranje sa dimenzijama tenzora.

In [9]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(
            self, 
            model_dimension: int, 
            heads: int, 
            dropout: float
        ) -> None:
        super().__init__()

        self.model_dimension = model_dimension
        self.heads = heads
        self.dropout = nn.Dropout(dropout)

        assert model_dimension % heads == 0, "model_dimension is not divisible by the number of heads."

        self.head_dimension = model_dimension // heads

        self.w_q = nn.Linear(model_dimension, model_dimension)
        self.w_k = nn.Linear(model_dimension, model_dimension)
        self.w_v = nn.Linear(model_dimension, model_dimension)

        self.w_o = nn.Linear(model_dimension, model_dimension)

    @staticmethod
    def attention(
            query, 
            key, 
            value, 
            mask, 
            dropout: nn.Dropout
        ):
        head_dimension = query.shape[-1]

        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(head_dimension)

        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)

        attention_scores = attention_scores.softmax(dim = -1)

        if dropout is not None:
            attention_scores = dropout(attention_scores)

        return (attention_scores @ value), attention_scores
    
    def forward(self, q, k, v, mask):

        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.shape[0], query.shape[1], self.heads, self.head_dimension).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.heads, self.head_dimension).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.heads, self.head_dimension).transpose(1, 2)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
         
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.heads * self.head_dimension)

        return self.w_o(x)

### Feed forward neural network

Feed forward je najprostija implementacija neuralne mreže. Naziv označava samo to da podaci idu u jednom smeru (od input-a ka output-u), i da nema drugih veza u mreži.

U encoder-u, sastoji se iz samo jednog skrivenog sloja, sa $d_ff$ čvorova. Ona je potpuno povezana. Ulaz u nju je jedan vektor izlaza iz multi-head attention-a, a izlaz je vektor iste te dimenzije. (Dakle, radi posebno sa "output" embedding-ovima tokena iz self attention-a!)

Njena uloga je (koliko ja razumem) dvostruka:

1. Primećujemo da multi-head self attention zapravo samo "meša" podatke. Transformacija koja se dobije na kraju tog bloka je samo linearna kombinacija početnih embedding-a. *FNN* je način da unesemo nelinearnost među podatke.

2. Omogućavamo da sada token primi kontekst samo od samog sebe, za razliku od ranije, kada je primao kontekst i od svih drugih tokena.

Novi modeli su samo dve matrice, redom dimenzija $d_{model} \times d_{ff}$ i $d_{ff} \times d_{model}.$

Implementacija je još jednom samo primena formule.

In [10]:
class FeedForwardBlock(nn.Module):

    def __init__(
            self, 
            model_dimension: int, 
            feed_forward_dimension: int, 
            dropout: float
        ) -> None:
        super().__init__()
        
        self.linear_1 = nn.Linear(model_dimension, feed_forward_dimension)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(feed_forward_dimension, model_dimension)

    def forward(self, x):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

### Sklapanje encoder-a

Sada su tu svi koraci potrebni da bi se sklopio encoder. 

U kodu je potrebna još jedna stvar. Blokovi u kojima se radi normalizacija zapravo ulaz u prethodni blok sabiraju sa izlazom iz prethodnog bloka za normalizovani ulaz. (U radu "Attention is all you need", ovo je implementirano obrnuto, izlaz za normalan ulaz je normalizovan, ali empirijski se pokazalo da obrnuti redosled daje bolje rezultate.)

To jest, ako je $I$ ulaz u prethodni blok, a $O(I)$ izlaz iz njega, izlaz iz bloka Add & Norm je $I + O(\text{norm}(I)).$ (U radu je formula $\text{norm}(I + O(I))$.)

To se postiže na sledeći način:

In [11]:
class ResidualConnection(nn.Module):

    def __init__(
            self, 
            features: int, 
            dropout: float
        ) -> None:
        super().__init__()

        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

Encoder se zapravo sastoji iz više blokova. Taj broj označavamo sa $N_x$. Jedan blok encoder-a se sastoji iz Self-Attention bloka (zajedno sa svojim Add & Norm blokom), kao i Feed-Forward bloka (zajedno sa svojim Add & Norm) blokom.

To znači da se blok encoder-a implementira na sledeći način:

In [12]:
class EncoderBlock(nn.Module):

    def __init__(
            self, 
            features: int, 
            self_attention_block: MultiHeadAttentionBlock, 
            feed_forward_block: FeedForwardBlock, 
            dropout: float
        ) -> None:
        super().__init__()

        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block

        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, source_mask):

        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, source_mask))

        x = self.residual_connections[1](x, self.feed_forward_block)

        return x

Sve što je ostalo je da spojimo više encoder blokova u encoder. Treba da spojimo $N_x$ prethodnih blokova u jedan, i normalizujemo izlaz koji dobijemo nakon toga.

In [13]:
class Encoder(nn.Module):

    def __init__(
            self, 
            features: int, 
            layers: nn.ModuleList
        ) -> None:
        super().__init__()

        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)

        return self.norm(x)

Sada imamo potpun encoder. Izlaz iz encoder-a je deo ulaza u multi-headed cross attention blok decoder-a, kao što ćemo videti i objasniti u nastavku.

## Decoder

U decoder-u se, za razliku od encoder-a, obrađuje ono što bi trebalo da bude izlaz modela. Služi da procesira tokene jezika na koji hoćemo da prevedemo originalni tekst.

Svi blokovi koji se koriste u jednom bloku decoder-a su već definisani. Za razliku od encoder-a, imamo još samo jedan blok, a to je multi-head cross attention.

Prvi blok je opet multi-head self attention, zatim multi-head cross attention, i konačno feed forward.

Takođe, razlikuje se i ulaz u decoder. U ovom slučaju, ne dodajemo token ***[EOS]*** na kraj rečenice, već je ulaz nešto oblika ***[SOS] BRZI VOZ 271 DOLAZI KOLOSEK 7 [PAD] ... [PAD]***, dok želimo da konačni izlaz iz transformer-a bude ***BRZI VOZ 271 DOLAZI KOLOSEK 7 [EOS] [PAD] ... [PAD]***.

Dakle, ulaz u decoder prvo prolazi kroz multi-headed self attention, pa zatim u multi-headed cross attention (zajedno sa izlazima iz encoder-a), i na kraju kroz feed forward. Sve to prati i normalizacija, na isti način kao i u encoder-u.

Treba još obratiti pažnju na maskiranje u dekoderu. U cross attention-u ćemo koristiti ponovo *padded masking*.

### Causal masking

Svrha našeg modela je da prevede rečenicu na drugi jezik, ali koristeći samo rečenicu iz originalnog jezika. Zato je važno da tokeni u decoder-u (to jest, tokeni jezika na koji prevodimo) ne vide tokene koji dolaze nakon njih.

To se takođe postiže maskiranjem, baš kao i ranije. I radi se na isti način, samo je sada matrica za maskiranje drugačija. Da bismo poništili dejstvo tokena koji slede, logično je da to treba da bude trougaona matrica.

Na početku je bila definisana funkcija `causal_mask(size)`, koja služi da bismo dobili ovu matricu.

In [14]:
def causal_mask(size: int) -> torch.Tensor:
    
    mask = torch.triu(torch.ones(1, size, size), diagonal = 1).type(torch.int)
    return mask == 0

### Multi-head cross attention

Ono što ovaj deo radi se ne razlikuje od multi-head self attention-a. Jedino što se razlikuje je ulaz u njega.

Da se radi o self attention-u, ulaz bi bio izlaz iz prethodnog bloka i to i za $Q$, i za $K$ i za $V$ matrice.

Ovde, međutim, to je ulaz samo za $V$. Ulaz za $Q$ i $K$ je izlaz dobijen iz encoder-a. Sva matematika ostaje ista, i ništa se zapravo ne menja. Možemo da koristimo isti blok definisan od ranije.

### Sklapanje decoder-a

Slično kao malopre, blok decoder-a se implementira na sledeći način, dok decoder i dalje ima $N_x$ blokova.

In [15]:
class DecoderBlock(nn.Module):

    def __init__(
            self, 
            features: int, 
            self_attention_block: MultiHeadAttentionBlock, 
            cross_attention_block: MultiHeadAttentionBlock, 
            feed_forward_block: FeedForwardBlock, 
            dropout: float
        ) -> None:
        super().__init__()

        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, source_mask, target_mask):
        
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, target_mask))

        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, source_mask))

        x = self.residual_connections[2](x, self.feed_forward_block)

        return x

In [16]:
class Decoder(nn.Module):

    def __init__(
            self, 
            features: int, 
            layers: nn.ModuleList
        )-> None:
        super().__init__()

        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, source_mask, target_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, source_mask, target_mask)

        return self.norm(x)

## Transformer

Izlaz iz decoder-a još treba da nekako prebacimo nazad u tokene pa u stringove, jer smo dobili izlaz u obliku embeddinga.

To se postiže još jednim linearnim slojem (praktično projekcijom na vektorski prostor), gde je prva dimenzija $d_{model}$ a druga veličina rečnika.

Nakon toga, treba da izaberemo koji od tokena će biti sledeći, što opet radimo koristeći **softmax** funkciju, jer dobijeni vektor prvo prebacimo u verovatnoće koristeći je, a zatim odaberemo onu koordinatu sa najvećom verovatnoćom.

In [17]:
class ProjectionLayer(nn.Module):

    def __init__(
            self, 
            model_dimension: int, 
            vocab_size: int
        ) -> None:
        super().__init__()

        self.proj = nn.Linear(model_dimension, vocab_size)

    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim = -1)

Sada imamo sve što je potrebno da napravimo i klasu za transformer.

In [18]:
class Transformer(nn.Module):
    
    def __init__(
            self, 
            encoder: Encoder, 
            decoder: Decoder, 
            source_embed: InputEmbeddings, 
            target_embed: InputEmbeddings, 
            source_pos: PositionalEncoding, 
            target_pos: PositionalEncoding, 
            projection_layer: ProjectionLayer
        ) -> None:
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.source_embed = source_embed
        self.target_embed = target_embed
        self.source_pos = source_pos
        self.target_pos = target_pos
        self.projection_layer = projection_layer

    def encode(self, source, source_mask):
        source = self.source_embed(source)
        source = self.source_pos(source)
        return self.encoder(source, source_mask)
    
    def decode(self, encoder_output, source_mask, target, target_mask):
        target = self.target_embed(target)
        target = self.target_pos(target)
        return self.decoder(target, encoder_output, source_mask, target_mask)
    
    def project(self, x):
        return self.projection_layer(x)

Konačno, treba i da umemo da prosledimo sve parametre. To znači da inicijalizujemo sve parametre modela i napravimo sve neophodne blokove za model.

In [19]:
def build_transformer(
        source_vocab_size: int, 
        target_vocab_size: int, 
        source_context_size: int, 
        target_context_size: int, 
        model_dimension: int = 512, 
        number_of_blocks: int = 6, 
        heads: int = 8, 
        dropout: float = 0.1, 
        feed_forward_dimension: int = 2048
    ) -> Transformer:
    
    source_embed = InputEmbeddings(model_dimension, source_vocab_size)
    target_embed = InputEmbeddings(model_dimension, target_vocab_size)

    source_pos = PositionalEncoding(model_dimension, source_context_size, dropout)
    target_pos = PositionalEncoding(model_dimension, target_context_size, dropout)

    encoder_blocks = []
    for _ in range(number_of_blocks):
        encoder_self_attention_block = MultiHeadAttentionBlock(model_dimension, heads, dropout)
        feed_forward_block = FeedForwardBlock(model_dimension, feed_forward_dimension, dropout)
        encoder_block = EncoderBlock(model_dimension, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    decoder_blocks = []
    for _ in range(number_of_blocks):
        decoder_self_attention_block = MultiHeadAttentionBlock(model_dimension, heads, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(model_dimension, heads, dropout)
        feed_forward_block = FeedForwardBlock(model_dimension, feed_forward_dimension, dropout)
        decoder_block = DecoderBlock(model_dimension, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    encoder = Encoder(model_dimension, nn.ModuleList(encoder_blocks))
    decoder = Decoder(model_dimension, nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(model_dimension, target_vocab_size)

    transformer = Transformer(encoder, decoder, source_embed, target_embed, source_pos, target_pos, projection_layer)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [20]:
def get_model(
        config, 
        source_vocab_size: int, 
        target_vocab_size: int
    ) -> Transformer:
    
    model = build_transformer(
        source_vocab_size = source_vocab_size, 
        target_vocab_size = target_vocab_size, 
        source_context_size = config['context_size'], 
        target_context_size = config['context_size'], 
        model_dimension = config['model_dimension']
        )
    
    return model

### Trening

Konačno, treba da vidimo kako treniramo model. Za početak, pretpostavljamo da imamo bazu podataka, u kojoj imamo praktično dve kolone, a redovi su popunjeni parom rečenica na dva jezika.

*Learning rate* je hiperparametar koji kontroliše koliko *loss* funkcija utiče na promenu parametara. Međutim, ispostavilo se da njegovo smanjivanje tokom treninga pozitivno utiče na trening. To je i logično, jer learning rate predstavlja dužinu "koraka" za koji se funkcija spusti, pa što više treniramo, to smo bliže minimumu, i to manje moramo da se pomeramo da bismo ga "pogodili". Ovo se postiže korišćenjem optimizer-a, a to je u našem slučaju `torch.optim.Adam(model_parameters, learning_rate, ...)`.

Postoji mogućnost čuvanja i učitavanja već treniranih weight-ova modela, ukoliko treniranje ide iz više koraka, ili ukoliko je potrebno da se pristupi modelu za inferencu.

#### Loss funkcija

*Loss* funkcija koju model koristi je Cross Entropy Loss.

Videli smo malopre da je ulaz u decoder ***[SOS] BRZI VOZ 271 DOLAZI KOLOSEK 7 [PAD] ... [PAD]***, i da želimo da konačni izlaz iz transformer-a bude ***BRZI VOZ 271 DOLAZI KOLOSEK 7 [EOS] [PAD] ... [PAD]***. Ovo znači da će naša *loss* funkcija uporediti ovaj željeni izlaz sa dobijenim izlazom. Ova konkretna *loss* funkcija se računa po formuli:

$$L(\mathbf{y}, \hat{\mathbf{y}}) = -\sum_{i=1}^{\text{vocab size}} y_i \log (\hat{y}_i).$$

Varijable u prošloj formuli su sledeće:

1. $\mathbf{y}$ predstavlja vektor dužine `vocab_size`, gde je jedinica na koordinati tačnog tokena, a sve ostalo nule.

2. $\hat{\mathbf{y}}$ predstavlja izlaz iz modela u obliku verovatnoća za konkretan token.

3. $y_i$ i $\hat{y}_i$ su $i$-te koordinate ovih vektora.

Konačno, kada saberemo ove formule primenjene na svaki vektor za ulaz i izlaz decoder-a, i rezultat podelimo sa njihovim brojem, dobijamo formulu:

$$L(\mathbf{Y}, \hat{\mathbf{Y}}) = -\frac{1}{N} \sum_{n = 1}^N \sum_{i = 1}^{\text{vocab size}} y_{n, i} \log (\hat{y}_{n, i}).$$

U ovom slučaju $\mathbf{Y}$ i $\hat{\mathbf{Y}}$ su matrice gde ima $N$ redova i $\text{vocab size}$ kolona, a svaki red je odgovarajući token u ulazu, odnosno izlazu.

Dakle, tokom treninga stalno računamo vrednost *loss* funkcije, nakon računanja ažuriramo weight-ove tako da u sledećem koraku *loss* bude, nadamo se, smanjen. Weight-ovi se ažuriraju korišćenjem backpropagation-a, kao i u svakom ML modelu.

Osim toga, svaki korak modela se sastoji iz jednog batch-a, što je način da model radi sa više ulaza odjednom, čime paralelizujemo treniranje i ubrzavamo model (ali ne linearno!).

Pre nego što definišemo model, treba i da znamo koji fajl, u situaciji da postoji, treba da pročitamo da bismo dobili weight-ove modela.

In [21]:
def get_weights_file_path(
        config, 
        epoch: str
    ) -> str:
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"

    return str(Path('.') / model_folder / model_filename)

def get_latest_weights(config) -> str:

    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}*"
    model_filenames = list(Path(model_folder).glob(model_filename))

    if len(model_filenames) == 0:
        return None
    
    def extract_epoch(filename):
        return int(filename.stem.split('_')[-1])
    
    model_filenames.sort(key = extract_epoch)

    return str(model_filenames[-1])

Takođe, moramo i bazu da dostavimo u odgovarajućoj formi. U našem slučaju, koristimo prvo funkciju `load_data(source_language, target_language)` da učitamo `{source_language}-{target_language}.tmx` fajl, a zatim funkciju `get_dataset(config)` da bismo definisali trening, test i validacijske skupove, kao i način za njihovo učitavanje.

In [22]:
def load_data(
        source_language: str, 
        target_language: str
    ) -> HFDataset:
    with open(f"{source_language}-{target_language}.tmx", "rb") as fin:
        tmx_file = tmxfile(fin, "en", "sr_Cyrl")

    data = {'id' : [], 'translation': []}
    i = 0

    for item in tmx_file.unit_iter():

        data["id"].append(str(i))
        i = i + 1

        data["translation"].append({f"{source_language}": item.source.strip('"\n').lower(), f"{target_language}": item.target.strip('"\n').lower()})

    dataset = HFDataset.from_dict(data)

    return dataset

In [23]:
def get_dataset(config):
    dataset_raw = load_data(config['source_language'], config['target_language'])

    dataset_size = len(dataset_raw)
    train_dataset_size = int(0.9 * dataset_size)
    validation_dataset_size = int(0.08 * dataset_size)
    test_dataset_size = dataset_size - train_dataset_size - validation_dataset_size

    training_dataset_raw, validation_dataset_raw, test_dataset_raw = random_split(dataset_raw, [train_dataset_size, validation_dataset_size, test_dataset_size])

    source_tokenizer = get_or_build_tokenizer(config, training_dataset_raw, config['source_language'], force_rewrite = True)
    target_tokenizer = get_or_build_tokenizer(config, training_dataset_raw, config['target_language'], force_rewrite = True)

    training_dataset = BilingualDataset(training_dataset_raw, source_tokenizer, target_tokenizer, config['source_language'], config['target_language'], config['context_size'])
    validation_dataset = BilingualDataset(validation_dataset_raw, source_tokenizer, target_tokenizer, config['source_language'], config['target_language'], config['context_size'])
    test_dataset = BilingualDataset(test_dataset_raw, source_tokenizer, target_tokenizer, config['source_language'], config['target_language'], config['context_size'])

    training_dataloader = DataLoader(training_dataset, batch_size = config['batch_size'], shuffle = True)
    validation_dataloader = DataLoader(validation_dataset, batch_size = 1, shuffle = True)
    test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle = True)

    return training_dataloader, validation_dataloader, test_dataloader, source_tokenizer, target_tokenizer

In [24]:
def train_model(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device {device}.')

    Path(config['model_folder']).mkdir(parents = True, exist_ok = True)

    training_dataloader, validation_dataloader, test_dataloader, source_tokenizer, target_tokenizer = get_dataset(config)
    model = get_model(config, source_tokenizer.get_vocab_size(), target_tokenizer.get_vocab_size()).to(device)

    writer = SummaryWriter(config['experiment_name'])

    optimizer = torch.optim.Adam(model.parameters(), lr = config['learning_rate'], eps = 1e-9)

    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = get_latest_weights(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None

    if model_filename:
        print(f"Preloading model {model_filename}.")
        state = torch.load(model_filename)
        optimizer.load_state_dict(state['optimizer_state_dict'])
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        global_step = state['global_step']
    else:
        print("No model to preload, starting from the beginning.")

    loss_function = nn.CrossEntropyLoss(ignore_index = source_tokenizer.token_to_id('[PAD]'), label_smoothing = 0.1).to(device)

    for epoch in range(initial_epoch, config['num_epochs']):
        
        batch_iterator = tqdm(training_dataloader, desc = f"Processing epoch {epoch:02d}")
        for batch in batch_iterator:

            model.train()

            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)

            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            transformer_output = model.project(decoder_output)

            label = batch['label'].to(device)

            loss = loss_function(transformer_output.view(-1, target_tokenizer.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            writer.add_scalar('train_loss', loss.item(), global_step)
            writer.flush()

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            global_step += 1

        run_validation(model, validation_dataloader, source_tokenizer, target_tokenizer, config['context_size'], device, lambda msg: batch_iterator.write(msg), writer, global_step, number_examples = 1)

        model_filename = get_weights_file_path(config, f'{epoch:02d}')
        if epoch % 50 == 49 or epoch == 0 or epoch == config['num_epochs'] - 1:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step
            }, model_filename)
            
    run_validation(model, validation_dataloader, source_tokenizer, target_tokenizer, config['context_size'], device, lambda msg: batch_iterator.write(msg), writer, global_step, number_examples = 50)
    run_test(test_dataloader)

U ovom kodu:

1. `writer` je način za vizualizaciju podataka. Njega ćemo videti ponovo u validaciji.

2. `loss.backward()` računa izvode i zapisuje ih u sve parametre za koje je `require_grad = True`, a onda `optimizer.step()` ažurira vrednosti tih parametara.

3. `run_validation` i `run_test` su funkcije koje ćemo kasnije implementirati, a koje obavljaju validaciju i testiranje modela.

#### Validacija

Validacija generalno služi za podešavanje hiperparametara modela. 

U slučaju transformera, validacija radi na sledeći način. Pretpostavimo da imamo rečenicu ***Voz za Kraljevo kasni.*** nad kojom hoćemo da izvršimo validaciju. Neka je njen tačan prevod ***VOZ DO KRALJEVO KASNI***.

1. Izvršimo encoder i dobijemo odgovarajući izlaz za datu rečenicu.

2. U decoder ubacujemo samo ***[SOS]***. Izlaz treba da bude neki token $T_1^1$.

3. U decoder ubacujemo ***[SOS] VOZ***. Izlaz treba da budu dva tokena ***VOZ*** $T_2^2$.

4. U decoder ubacujemo ***[SOS] VOZ DO***. Izlaz treba da budu tri tokena ***VOZ DO*** $T_3^3$.

5. Ovaj proces ponavljamo dok ne prođemo celu rečenicu.

Dakle, validiramo kao da imamo više malih rečenica, i uvek predviđamo po tačno $1$ sledeći token.

Zato su nam potrebne dve funkcije, a to su:

1. Predviđanje sledećeg tokena, na osnovu treniranog modela i izlaza iz encoder-a - `_greedy_decode_next_token(model, encoder_output, decoder_input, source_mask, device)`;

2. Funkcija `run_validation(model, validation_dataset, source_tokenizer, target_tokenizer, max_length, device, print_msg, writer, global_step, number_examples)` od malopre, koja služi da bismo tokom treniranja mogli da radimo validaciju modela.

Takođe, definisaćemo i funkciju `_greedy_decode(model, encoder_input, source_mask, source_tokenizer, target_tokenizer, max_length, device)`, koja služi da predvidi ceo izlaz, počevši sa tokenom $\left[\text{SOS}\right]$. Tokom testiranja ćemo videti zašto je ona bitna.

In [24]:
def _greedy_decode(
        model: Transformer, 
        encoder_input: torch.Tensor, 
        source_mask: torch.Tensor, 
        source_tokenizer: Tokenizer, 
        target_tokenizer: Tokenizer, 
        max_length: int, 
        device: str
    ) -> torch.Tensor:
    sos_index = target_tokenizer.token_to_id('[SOS]')
    eos_index = target_tokenizer.token_to_id('[EOS]')

    encoder_output = model.encode(encoder_input, source_mask)

    decoder_input = torch.empty(1, 1).fill_(sos_index).type_as(encoder_input).to(device)

    while True:

        if decoder_input.size(1) == max_length:
            break

        decoder_input, next_token = _greedy_decode_next_token(model, encoder_output, decoder_input, source_mask, device)

        if next_token == eos_index:
            break

    return decoder_input.squeeze(0)


def _greedy_decode_next_token(
        model: Transformer, 
        encoder_output: torch.Tensor,
        decoder_input: torch.Tensor,
        source_mask: torch.Tensor, 
        device: str
    ) -> Tuple[torch.Tensor, torch.Tensor]:

    decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)
    out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

    prob = model.project(out[:, -1])
    _, next_token = torch.max(prob, dim = 1)

    decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(decoder_input).fill_(next_token.item()).to(device)], dim = 1)

    return decoder_input, next_token


def run_validation(
        model: Transformer, 
        validation_dataset: DataLoader, 
        source_tokenizer: Tokenizer, 
        target_tokenizer: Tokenizer, 
        max_length: int, 
        device: str, 
        print_msg, 
        writer: SummaryWriter, 
        global_step: int, 
        number_examples: int = 2
    ) -> None:
    model.eval()

    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        with os.open("stty size", "r") as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():

        for batch in validation_dataset:

            source_text = batch['source_text'][0]
            print_msg('-' * console_width)
            print_msg(f"Source: {source_text}")

            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            sos_index = target_tokenizer.token_to_id('[SOS]')

            target_ids = target_tokenizer.encode(batch['target_text'][0]).ids

            next_token = None

            decoder_input_slice = torch.empty(1, 1).fill_(sos_index).type_as(encoder_input).to(device)

            encoder_output = model.encode(encoder_input, encoder_mask)

            next_tokens_predicted = []
            next_tokens_actual = []

            i = 1
            while i <= len(target_ids):

                _, next_token = _greedy_decode_next_token(model, encoder_output, decoder_input_slice, encoder_mask, device)

                next_token = next_token.squeeze(0).detach().cpu().numpy()
                if next_token.ndim == 0:
                    next_token = [next_token.item()]
                elif next_token.ndim == 1:
                    next_token = next_token.tolist()

                next_tokens_predicted.append(target_tokenizer.decode(next_token))
                next_tokens_actual.append(target_tokenizer.decode([torch.tensor(target_ids[i - 1]).unsqueeze(0).to(device).squeeze(0).detach().cpu().numpy()]))

                decoder_input_slice = torch.tensor(target_ids[0 : i]).unsqueeze(0).to(device)

                i += 1

            table = [next_tokens_predicted, next_tokens_actual]
            print_msg(tabulate(table, headers = 'keys', showindex = True, tablefmt = 'grid'))

            model_out = _greedy_decode(model, encoder_input, encoder_mask, source_tokenizer, target_tokenizer, max_length, device)

            target_text = batch['target_text'][0]

            model_out_text = target_tokenizer.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            print_msg(f"Target: {target_text}")
            print_msg(f"Predicted: {model_out_text}")

            if count == number_examples:
                print_msg('-' * console_width)
                break

    if writer:

        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()

### Testiranje

Konačno, da bismo testirali model, treba da mu na neki način dozvolimo da generiše celu rečenicu.

To postižemo na sličan način kao u validaciji. Pretpostavljamo opet isti ulaz *Voz za Kraljevo kasni.* Model u početku ne zna za ovu rečenicu, nije bila u trening skupu, pa nemamo nikakav unapred poznat ulaz u decoder.

1. Izvršimo encoder i dobijemo odgovarajući izlaz za datu rečenicu.

2. U decoder ubacujemo samo $\left[\text{SOS}\right]$. Izlaz treba da bude neki token $T_1$.

3. U decoder ubacujemo $\left[\text{SOS}\right]\ T_1$. Izlaz treba da budu dva tokena $T_1\ T_2$.

4. U decoder ubacujemo $\left[\text{SOS}\right]\ T_1\ T_2$. Izlaz treba da budu tri tokena $T_1\ T_2\ T_3$.

5. Ovaj proces ponavljamo dok ne dođemo do izlaza $T_1\ T_2\ T_3\ \dots\ T_n\ \left[\text{EOS}\right]$.

Dakle, model generiše jedan po jedan token, ažurirajući svoj ulaz u decoder, sve dok ne dobijemo token $\left[\text{EOS}\right]$ u izlazu.

Da bismo to uradili, potrebno nam je još funkcija:

1. `_prepare_model(config)` koja priprema sve neophodne karakteristike modela;

2. `_translate_sentence(model, sentence, source_tokenizer, target_tokenizer, max_length, device, pad_token, predict_next_tokens)` koja sa datim karakteristikama modela prevodi zadatu rečenicu;

3. `translate_sentence(sentence)` koja je user-friendly wrapper za funkciju `_translate_sentence` (jer kao ulaz uzima samo string koji predstavlja rečenicu koju treba prevesti);

4. `translate_sentences(sentences)` koja prevodi listu rečenica, na sličan način kao i prethodna funkcija;

5. `run_test(test_dataset)` koja prevodi rečenice iz test baze (koja je odvojena pre treniranja modela).

In [ ]:
def _prepare_model(config):
    source_language = config['source_language']
    target_language = config['target_language']

    source_tokenizer = Tokenizer.from_file(str(config['tokenizer_file'].format(source_language)))
    target_tokenizer = Tokenizer.from_file(str(config['tokenizer_file'].format(target_language)))

    max_length = config['context_size']

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = get_model(config, source_tokenizer.get_vocab_size(), target_tokenizer.get_vocab_size()).to(device)
    model_filename = get_latest_weights(config)
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])

    pad_token = torch.tensor([source_tokenizer.token_to_id('[PAD]')], dtype = torch.int64).to(device)

    return {
        'model': model,
        'source_tokenizer': source_tokenizer,
        'target_tokenizer': target_tokenizer,
        'max_length': max_length,
        'device': device,
        'pad_token': pad_token
    }

def _translate_sentence(
        model: Transformer,
        sentence: str,
        source_tokenizer: Tokenizer,
        target_tokenizer: Tokenizer,
        max_length: int,
        device: str,
        pad_token: torch.Tensor,
        predict_next_tokens: Callable[[Transformer, torch.Tensor, torch.Tensor, Tokenizer, Tokenizer, int, str], torch.Tensor] = _greedy_decode
    ) -> str:
    model.eval()

    with torch.no_grad():

        source_ids = source_tokenizer.encode(sentence).ids
        encoder_input = torch.tensor(source_ids).unsqueeze(0).to(device)

        source_mask = (encoder_input != pad_token).unsqueeze(0).unsqueeze(0).int().to(device)

        model_output = predict_next_tokens(
            model,
            encoder_input,
            source_mask,
            source_tokenizer,
            target_tokenizer,
            max_length,
            device
        )

        translated_sentence = target_tokenizer.decode(model_output.detach().cpu().numpy())

    model.train()

    return translated_sentence

def translate_sentence(sentence: str) -> str:
    config = get_config()

    model_parameters = _prepare_model(config)

    model = model_parameters['model']
    source_tokenizer = model_parameters['source_tokenizer']
    target_tokenizer = model_parameters['target_tokenizer']
    max_length = model_parameters['max_length']
    device = model_parameters['device']
    pad_token = model_parameters['pad_token']

    translation = _translate_sentence(
        model,
        sentence,
        source_tokenizer,
        target_tokenizer,
        max_length,
        device,
        pad_token
    )

    return translation

def translate_sentences(sentences: List[str]) -> List[str]:
    config = get_config()

    model_parameters = _prepare_model(config)

    model = model_parameters['model']
    source_tokenizer = model_parameters['source_tokenizer']
    target_tokenizer = model_parameters['target_tokenizer']
    max_length = model_parameters['max_length']
    device = model_parameters['device']
    pad_token = model_parameters['pad_token']

    translations = []
    for sentence in sentences:
        translations.append((sentence, _translate_sentence(
            model,
            sentence,
            source_tokenizer,
            target_tokenizer,
            max_length,
            device,
            pad_token
            )))
        
    return translations


def run_test(test_dataset: DataLoader):
    
    sentences = []

    for batch in test_dataset:

        source_text = batch['source_text'][0]
        sentences.append(source_text)

    translations = translate_sentences(sentences)
    for translation in translations:
        print(f"Original: {translation[0]}")
        print(f"Translated: {translation[1]}")